### Importing libraries

In [1]:
import pymongo
from pymongo import MongoClient
import json
import requests
from bson.code import Code

#### Establishing connetion with mongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017/")

# Reads "reviews_electronics.16.json" and uploads each review as a separate document to the collection "reviews" in the database "amazon".

#### Creating a database called 'amazon'

In [3]:
amazon_db = client["amazon"]

#### Creating a collection called "reviews"

In [4]:
reviews_collection = amazon_db["reviews"]

#### Loading the data into a list of dictionaries

In [21]:
reviews_list = []
for line in open('reviews_electronics.16.json', 'r'):
    reviews_list.append(json.loads(line))

In [22]:
type(reviews_list)

list

#### Saving the data into mongoDB amazon database reviews collection

In [23]:
reviews_collection.insert_many(reviews_list)

In [25]:
reviews_list[3]

{'reviewerID': 'ATM6RRB90D8DV',
 'asin': 'B00E4KP4W6',
 'reviewerName': 'Matthew D. Irmen "Jazz Fan"',
 'helpful': [1, 2],
 'reviewText': "I bought this screen protector because I was unimpressed with the Otterbox Defender attached screen protector. If I have a retina display iPad, I want it to look like it has retina display, not a hazy iPad 2. So... I started researching and came across this tempered glass screen protector with promises of easy installation and beautiful retina quality display. I have to admit that I am very impressed with the picture quality and the ease of installation. Best of all, I was able to tear out the plastic screen protector from my Otterbox defender case and use the case without issue. So why did I give it 4 stars? Because of the home button sticker! The sticker is a lame plastic blob that sits on the home button and looks ugly. I thought little of it because I would bury it under the frame of the Otterbox case. Unfortunately it causes slight problems wit

### Uses MongoDB's map reduce function to build a new collection "avg_scores" that averages review scores by product ("asin"). Print the first 100 entries of "avg_scores" to screen.

#### Making the Map function for finding average

In [27]:
map_1 = Code( "function () { emit(this.asin, this.overall) }")

#### Making the Reduce function for finding average

In [28]:
reduce_1 = Code("function(asin, overall) { return Array.avg(overall) }")

#### Running the MapReduce function

In [29]:
results_1 = reviews_collection.map_reduce(map_1, reduce_1, out="avg_scores")

#### Checking the collection names in the database

In [30]:
amazon_db.collection_names()

C:\Users\mites\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['reviews', 'avg_scores']

In [58]:
resul1_first100 = amazon_db.avg_scores.find({}).limit(100)

#### Printing the first 100 averages

In [59]:
for x in resul1_first100:
    print('Product = {0} \n  Average Score = {1}\n'.format(x['_id'], x['value']))

Product = 0132793040 
  Average Score5.0

Product = B00E4KP4W6 
  Average Score4.545454545454546

Product = B00E4KP8VI 
  Average Score5.0

Product = B00E4KPMC8 
  Average Score2.0

Product = B00E4KQ5C4 
  Average Score5.0

Product = B00E4KQ9GG 
  Average Score3.2857142857142856

Product = B00E4KQ9K2 
  Average Score5.0

Product = B00E4KQD4E 
  Average Score4.0

Product = B00E4KZBX8 
  Average Score4.0

Product = B00E4KZDJ0 
  Average Score5.0

Product = B00E4L35DA 
  Average Score4.0

Product = B00E4L3N9Q 
  Average Score4.0

Product = B00E4L48EA 
  Average Score5.0

Product = B00E4L7FLI 
  Average Score1.0

Product = B00E4L7TS2 
  Average Score4.0

Product = B00E4LAL82 
  Average Score3.0

Product = B00E4LBZZK 
  Average Score5.0

Product = B00E4LF2Z4 
  Average Score4.333333333333333

Product = B00E4LFP0G 
  Average Score4.444444444444445

Product = B00E4LFWWW 
  Average Score4.4

Product = B00E4LGTVU 
  Average Score4.195658625514055

Product = B00E4LGTXS 
  Average Score3.92307692

### Uses MongoDB's map reduce function to build a new collection "weighted_avg_scores" that averages review scores by product ("asin"), weighted by the number of helpful votes (The base weight is 1 and for every additional helpful vote add 1 to weight. e.g. a "[3, 5]" value on "helpful" column should use 3 + 1 = 4 as weight, 3 being the additional votes and 1 being the base weight). Print the first 100 entries of "weighted_avg_scores" to screen.

#### Making the Map function for finding weighted average

In [5]:
map_2 = Code('''function(){
    var wtp1=this.helpful[0]+1;
    var value= {
        oa: wtp1*this.overall,
        wt: wtp1
    };
    emit(this.asin,value);
};''')

#### Making the Reduce function for finding weighted average

In [6]:
reduce_2 = Code('''function (key, values) {
    reducedVal= { oa: 0, wt: 0};
    for (var i=0; i<values.length; i++) {
        
        reducedVal.oa+=values[i].oa;
        reducedVal.wt+=values[i].wt;
    }
    return reducedVal;
};''')

#### Making the Finalize function for finding weighted average

In [9]:
finalize_2 = Code('''function (key, reducedVal) {
    reducedVal.wtavg= reducedVal.oa/reducedVal.wt;
    return reducedVal.wtavg;
};''')

#### Running the MapReduce function to calculate weighted average

In [10]:
results_2 = reviews_collection.map_reduce(map_2, reduce_2, out="weighted_avg_score", finalize=finalize_2)

#### Checking the collection names in the database

In [13]:
amazon_db.collection_names()

C:\Users\mites\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['weighted_avg_score', 'reviews', 'avg_scores']

In [11]:
result2_first100 = amazon_db.weighted_avg_score.find({}).limit(100)

#### Printing the first 100 averages

In [12]:
for y in result2_first100:
    print('Product = {0} \n  Weighted Average Score = {1}\n'.format(y['_id'], y['value']))

Product = 0132793040 
  Weighted Average Score = 5.0

Product = B00E4KP4W6 
  Weighted Average Score = 4.684210526315789

Product = B00E4KP8VI 
  Weighted Average Score = 5.0

Product = B00E4KPMC8 
  Weighted Average Score = 2.0

Product = B00E4KQ5C4 
  Weighted Average Score = 5.0

Product = B00E4KQ9GG 
  Weighted Average Score = 3.6875

Product = B00E4KQ9K2 
  Weighted Average Score = 5.0

Product = B00E4KQD4E 
  Weighted Average Score = 4.0

Product = B00E4KZBX8 
  Weighted Average Score = 4.0

Product = B00E4KZDJ0 
  Weighted Average Score = 5.0

Product = B00E4L35DA 
  Weighted Average Score = 3.0

Product = B00E4L3N9Q 
  Weighted Average Score = 4.0

Product = B00E4L48EA 
  Weighted Average Score = 5.0

Product = B00E4L7FLI 
  Weighted Average Score = 1.0

Product = B00E4L7TS2 
  Weighted Average Score = 4.0

Product = B00E4LAL82 
  Weighted Average Score = 3.0

Product = B00E4LBZZK 
  Weighted Average Score = 5.0

Product = B00E4LF2Z4 
  Weighted Average Score = 4.15384615384615